In [1]:
!pip install opencv-python-headless
!pip install tensorflow

In [2]:
from google.colab import files
uploaded = files.upload()

# Assuming the video file is named 'traffic.mp4'

Saving traffic.mp4 to traffic.mp4


In [3]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [9]:
from google.colab.patches import cv2_imshow

In [4]:
# Load the SSD MobileNet V2 model from TensorFlow Hub
model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

In [5]:
def detect_objects(frame):
    # Convert frame to tensor
    input_tensor = tf.convert_to_tensor(frame, dtype=tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run detection
    detections = model(input_tensor)
    detection_classes = detections['detection_classes'][0].numpy().astype(int)
    detection_scores = detections['detection_scores'][0].numpy()
    detection_boxes = detections['detection_boxes'][0].numpy()

    return detection_classes, detection_scores, detection_boxes

In [6]:
def adjust_signal_timing(vehicle_count, base_time=30, max_time=120):
    if vehicle_count > 50:
        return max_time
    elif vehicle_count > 20:
        return base_time + (vehicle_count - 20) * 2
    else:
        return base_time

In [ ]:

# Load the uploaded video
video_path = "traffic.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB (required by the model)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect objects in the frame
    classes, scores, boxes = detect_objects(rgb_frame)
    vehicle_count = sum((scores > 0.5) & (classes == 3))  # '3' is the class ID for vehicles

    # Adjust the signal timing based on vehicle count
    current_timing = adjust_signal_timing(vehicle_count)
    print(f"Vehicle Count: {vehicle_count}, Adjusted Signal Timing: {current_timing} seconds")

    # Draw bounding boxes
    for i in range(len(scores)):
        if scores[i] > 0.5 and classes[i] == 3:
            box = boxes[i] * np.array([frame.shape[0], frame.shape[1], frame.shape[0], frame.shape[1]])
            (startY, startX, endY, endX) = box.astype("int")
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Display the frame with bounding boxes (for debugging purposes)
    cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow in Colab

cap.release()